In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib.pyplot import cm

In [2]:
import sys
print(sys.path) 
sys.path.append("/cellar/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages")
sys.path.insert(1, '/cellar/users/mpagadal/Data/gcta/scripts')

['/data/cellardata/users/mpagadal/projects/germline-immune/cell-type-specificity/notebooks', '/nrnb/opt/anaconda3.7/lib/python37.zip', '/nrnb/opt/anaconda3.7/lib/python3.7', '/nrnb/opt/anaconda3.7/lib/python3.7/lib-dynload', '', '/cellar/users/mpagadal/.local/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/IPython/extensions', '/cellar/users/mpagadal/.ipython']


In [3]:
import networkx as nx
from ndex2.nice_cx_network import NiceCXNetwork 
import ndex2.client as nc
import ndex2

In [4]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [5]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3

In [6]:
import statsmodels.stats.multitest as multi
from adjustText import adjust_text

In [7]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import json

## Get GTEx summary statistics for immune microenvironment SNPs from all tissue types

In [8]:
with open('/cellar/users/mpagadal/Data/resources/tcga/ensembl_map.json', 'r') as f:
    ensembl_dict = json.load(f)

In [9]:
ensembl_dict={k.split(".")[0]:v for k,v in ensembl_dict.items()}

In [10]:
def compile_gtex(directory,file_name,mapping):
    gtex_files=[x for x in os.listdir(directory)]
    total_gtex=pd.DataFrame()

    for x in gtex_files:
        cis=pd.read_csv(directory+x)
        cis["file"]=x.split(file_name)[0]
        total_gtex=total_gtex.append(cis)
    
    #format variant2 and gene
    total_gtex["variant2"]=total_gtex["variant_b37"].str.split("_").str[0]+":"+total_gtex["variant_b37"].str.split("_").str[1]+":"+total_gtex["variant_b37"].str.split("_").str[3]+":"+total_gtex["variant_b37"].str.split("_").str[2]
    #get gene ids for mapping
    total_gtex["gene_id"]=total_gtex["phenotype_id"].str.split(".").str[0]
    
    total_gtex["ref"]=total_gtex["variant_id"].str.split("_").str[2]
    total_gtex["alt"]=total_gtex["variant_id"].str.split("_").str[3]
    
    total_gtex_melt=pd.melt(total_gtex, id_vars=(["gene_id","slope","slope_se","ref","alt","file","pval_nominal","maf","tss_distance","variant_b37"]),value_vars=(["variant","variant2"]))
    
    total_gtex_melt["gene_name"]=total_gtex_melt["gene_id"].map(mapping)
    
    total_gtex_melt["assoc"]=total_gtex_melt["value"]+"_"+total_gtex_melt["gene_name"]
    
    return(total_gtex_melt)
    

In [156]:
gtex=compile_gtex("./rand_extracted_snps/",".snps.csv",ensembl_dict)

In [157]:
print(gtex.shape)
gtex.head()

(1579454, 14)


,gene_id,slope,slope_se,ref,alt,file,pval_nominal,maf,tss_distance,variant_b37,variable,value,gene_name,assoc
0,ENSG00000227232,0.043485,0.181287,A,C,Brain_Anterior_cingulate_cortex_BA24,0.810879,0.111111,801074,1_766007_A_C_b37,variant,1:766007:A:C,WASH7P,1:766007:A:C_WASH7P
1,ENSG00000186092,0.317116,0.176977,A,C,Brain_Anterior_cingulate_cortex_BA24,0.075907,0.111111,761536,1_766007_A_C_b37,variant,1:766007:A:C,OR4F5,1:766007:A:C_OR4F5
2,ENSG00000268903,-0.041768,0.154676,A,C,Brain_Anterior_cingulate_cortex_BA24,0.787640,0.111111,694732,1_766007_A_C_b37,variant,1:766007:A:C,RP11-34P13.15,1:766007:A:C_RP11-34P13.15
3,ENSG00000269981,-0.139097,0.159692,A,C,Brain_Anterior_cingulate_cortex_BA24,0.385634,0.111111,692662,1_766007_A_C_b37,variant,1:766007:A:C,RP11-34P13.16,1:766007:A:C_RP11-34P13.16
4,ENSG00000241860,-0.146531,0.198277,A,C,Brain_Anterior_cingulate_cortex_BA24,0.461467,0.111111,656765,1_766007_A_C_b37,variant,1:766007:A:C,RP11-34P13.13,1:766007:A:C_RP11-34P13.13


## Combine gtex information with TCGA association information

In [220]:
#get randomized genes
snps=pd.read_csv("gtex_rand.csv",delimiter="\t")
snps["gene_name"]=snps["gene_id"].map(ensembl_dict)
snps["assoc"]=snps["variant"]+"_"+snps["gene_name"]

In [221]:
snps_gtex=pd.merge(snps[["assoc"]],gtex,on="assoc",how="left")

In [223]:
snps_gtex["assoc"]=snps_gtex["assoc"].str.replace("-",".")

In [224]:
snps_gtex["assoc2"]=snps_gtex["value"].str.rsplit(":",2).str[0]+":"+snps_gtex["value"].str.split(":").str[3]+":"+snps_gtex["value"].str.split(":").str[2]+"_"+snps_gtex["gene_name"]

In [225]:
snps_gtex.columns

Index(['assoc', 'gene_id', 'slope', 'slope_se', 'ref', 'alt', 'file',
       'pval_nominal', 'maf', 'tss_distance', 'variant_b37', 'variable',
       'value', 'gene_name', 'assoc2'],
      dtype='object')

In [226]:
snps_gtex=pd.melt(snps_gtex,id_vars=(["variant_b37","tss_distance","slope","slope_se","ref","alt","file"]),value_vars=(["assoc","assoc2"]))

In [227]:
del snps_gtex["variable"]

In [228]:
snps_gtex=snps_gtex.rename(columns={"value":"assoc"})

In [229]:
len(snps_gtex["assoc"].unique())

964

## Identify of GTEx snps that were found that are cis associations

## See concordance between associations

In [230]:
tpm_cancer=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/snp-tables/brian.rand.gtex.tsv",delimiter="\t")
firebrowse_cancer=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/snp-tables/firebrowse.rand.gtex.tsv",delimiter="\t")
pancanatlas_cancer=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/snp-tables/pancanatlas.rand.gtex.tsv",delimiter="\t")

In [231]:
#get tissue mappings
tissue_mapping=pd.read_csv("gtex.tcga.tissue.csv")
tissue_mapping["combo"]=tissue_mapping["gtex tissues"]+"."+tissue_mapping["TCGA"]

In [232]:
tissue_mapping["whole_blood combo"]="Whole_Blood"+"."+tissue_mapping["TCGA"]

In [245]:
def make_bootstrap(df,name,combos):
    df["assoc"]=df["ID"]+"_"+df["pheno"]
    compare_beta=pd.merge(snps_gtex,df,on="assoc")
    
    compare_beta["combo"]=compare_beta["file"]+"."+compare_beta["file_name"]
    
    if combos == "regular":
        compare_beta=compare_beta[compare_beta["combo"].isin(tissue_mapping["combo"].tolist())]
    if combos == "blood":
        compare_beta=compare_beta[compare_beta["combo"].isin(tissue_mapping["whole_blood combo"].tolist())]
    
    for x in compare_beta["combo"].unique():
        compare_beta_combo=compare_beta[compare_beta["combo"]==x]
        #reverse slope for cases of ref/alt mismatch
        compare_beta_combo["slope"]=np.where(compare_beta_combo["alt"] != compare_beta_combo["ALT"], (compare_beta_combo["slope"]*-1),compare_beta_combo["slope"])
        print(x)
        print(compare_beta_combo.shape)
        compare_beta_combo["z"]=(compare_beta_combo["BETA"]-compare_beta_combo["slope"])/compare_beta_combo["slope_se"]
        compare_beta_combo.to_csv(name+"."+x+".csv")
    

In [250]:
make_bootstrap(tpm_cancer,"/cellar/users/mpagadal/Data/gtex/for_bootstrapping/brian_bootstrap/brian.rand","regular")
make_bootstrap(firebrowse_cancer,"/cellar/users/mpagadal/Data/gtex/for_bootstrapping/firebrowse_bootstrap/firebrowse.rand","regular")
make_bootstrap(pancanatlas_cancer,"/cellar/users/mpagadal/Data/gtex/for_bootstrapping/pancanatlas_bootstrap/pancanatlas.rand","regular")

/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Brain_Anterior_cingulate_cortex_BA24.GBM
(4620, 23)
Brain_Anterior_cingulate_cortex_BA24.LGG
(4620, 23)
Testis.TGCT
(4800, 23)
Brain_Hippocampus.GBM
(4630, 23)
Brain_Hippocampus.LGG
(4630, 23)
Brain_Hypothalamus.GBM
(4640, 23)
Brain_Hypothalamus.LGG
(4640, 23)
Brain_Amygdala.GBM
(4600, 23)
Brain_Amygdala.LGG
(4600, 23)
Brain_Cerebellum.GBM
(4670, 23)
Brain_Cerebellum.LGG
(4670, 23)
Brain_Cortex.GBM
(4650, 23)
Brain_Cortex.LGG
(4650, 23)
Pituitary.GBM
(4720, 23)
Pituitary.LGG
(4720, 23)
Colon_Transverse.COAD
(4680, 23)
Esophagus_Muscularis.ESCA
(4600, 23)
Lung.LUAD
(4690, 23)
Lung.LUSC
(4690, 23)
Adrenal_Gland.ACC
(4550, 23)
Stomach.STAD
(4640, 23)
Brain_Putamen_basal_ganglia.GBM
(4550, 23)
Brain_Putamen_basal_ganglia.LGG
(4550, 23)
Skin_Sun_Exposed_Lower_leg.SKCM
(4640, 23)
Prostate.PRAD
(4740, 23)
Minor_Salivary_Gland.HNSC
(4660, 23)
Brain_Nucleus_accumbens_basal_ganglia.GBM
(4580, 23)
Brain_Nucleus_accumbens_basal_ganglia.LGG
(4580, 23)
Esophagus_Mucosa.ESCA
(4580, 23)
Skin_Not_Sun_E

In [251]:
make_bootstrap(tpm_cancer,"/cellar/users/mpagadal/Data/gtex/for_bootstrapping/brian_bootstrap/brian.rand","blood")
make_bootstrap(firebrowse_cancer,"/cellar/users/mpagadal/Data/gtex/for_bootstrapping/firebrowse_bootstrap/firebrowse.rand","blood")
make_bootstrap(pancanatlas_cancer,"/cellar/users/mpagadal/Data/gtex/for_bootstrapping/pancanatlas_bootstrap/pancanatlas.rand","blood")

/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Whole_Blood.ACC
(4070, 23)
Whole_Blood.BRCA
(4080, 23)
Whole_Blood.COAD
(4070, 23)
Whole_Blood.ESCA
(4070, 23)
Whole_Blood.GBM
(4080, 23)
Whole_Blood.HNSC
(4080, 23)
Whole_Blood.KICH
(4070, 23)
Whole_Blood.KIRC
(4080, 23)
Whole_Blood.KIRP
(4070, 23)
Whole_Blood.LGG
(4070, 23)
Whole_Blood.LIHC
(4080, 23)
Whole_Blood.LUAD
(4070, 23)
Whole_Blood.LUSC
(4070, 23)
Whole_Blood.OV
(4070, 23)
Whole_Blood.PAAD
(4070, 23)
Whole_Blood.PRAD
(4080, 23)
Whole_Blood.SKCM
(4070, 23)
Whole_Blood.STAD
(4080, 23)
Whole_Blood.TGCT
(4080, 23)
Whole_Blood.THCA
(4080, 23)
Whole_Blood.UCEC
(4070, 23)
Whole_Blood.UCS
(4070, 23)
Whole_Blood.ACC
(4080, 23)
Whole_Blood.BRCA
(4080, 23)
Whole_Blood.COAD
(4070, 23)
Whole_Blood.ESCA
(4080, 23)
Whole_Blood.GBM
(4080, 23)
Whole_Blood.HNSC
(4080, 23)
Whole_Blood.KICH
(4080, 23)
Whole_Blood.KIRC
(4080, 23)
Whole_Blood.KIRP
(4080, 23)
Whole_Blood.LGG
(4080, 23)
Whole_Blood.LIHC
(4080, 23)
Whole_Blood.LUAD
(4080, 23)
Whole_Blood.LUSC
(4080, 23)
Whole_Blood.OV
(4080, 23)
Who

In [253]:
files=[x.split("brian.")[1] for x in os.listdir("/cellar/users/mpagadal/Data/gtex/for_bootstrapping/brian_bootstrap/")]

In [256]:
files=[x.replace("rand.","") for x in files]
files=[x.replace("test.","") for x in files]

In [263]:
len([x for x in set(files)])

75

In [265]:
direct=[]

for i in range(75):
    direct.append("brian")
for i in range(75):
    direct.append("firebrowse")
for i in range(75):
    direct.append("pancanatlas")

In [267]:
len(direct)

225